# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

## Pacotes

In [4]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

## Arquivos

In [6]:
filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"

# Carregar o arquivo de features
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")['nome_var']

# Carregar os rótulos
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

# Carregar os dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])['subject_id']
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

# Carregar os dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])['subject_id']
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:
%%time

# Criar e treinar a árvore de decisão com poda (ccp_alpha=0.001)
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Avaliar a acurácia
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)

# Exibir resultados
print(f"Acurácia no treino: {acc_train:.4f}")
print(f"Acurácia no teste: {acc_test:.4f}")

Acurácia no treino: 0.9758
Acurácia no teste: 0.8799
CPU times: total: 6.14 s
Wall time: 6.23 s


## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas um componente. Faça uma árvore de decisão com este componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [10]:
%%time

# Normalizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA (1 componente)
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Criar e treinar a árvore
clf_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=27)
clf_pca.fit(X_train_pca, y_train)

# Fazer previsões
y_train_pred_pca = clf_pca.predict(X_train_pca)
y_test_pred_pca = clf_pca.predict(X_test_pca)

# Avaliar a acurácia
acc_train_pca = accuracy_score(y_train, y_train_pred_pca)
acc_test_pca = accuracy_score(y_test, y_test_pred_pca)

# Exibir resultados
print(f"Acurácia no treino (PCA): {acc_train_pca:.4f}")
print(f"Acurácia no teste (PCA): {acc_test_pca:.4f}")
print(f"Variância explicada pelo 1º componente: {pca.explained_variance_ratio_[0]:.4f}")


Acurácia no treino (PCA): 0.4771
Acurácia no teste (PCA): 0.4316
Variância explicada pelo 1º componente: 0.5078
CPU times: total: 1.84 s
Wall time: 489 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [12]:
import time
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definir número de componentes a serem testados
num_components_list = [1, 2, 5, 10, 50]

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dicionário para armazenar resultados
results = []

# Testar diferentes quantidades de componentes principais
for num_components in num_components_list:
    start_time = time.time()  # Iniciar contagem de tempo
    
    # Aplicar PCA
    pca = PCA(n_components=num_components)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # Criar e treinar a árvore de decisão
    clf_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=27)
    clf_pca.fit(X_train_pca, y_train)

    # Fazer previsões
    y_train_pred = clf_pca.predict(X_train_pca)
    y_test_pred = clf_pca.predict(X_test_pca)

    # Avaliar a acurácia
    acc_train = accuracy_score(y_train, y_train_pred)
    acc_test = accuracy_score(y_test, y_test_pred)

    # Medir tempo de execução
    execution_time = time.time() - start_time

    # Armazenar resultados
    results.append([num_components, acc_train, acc_test, execution_time])

# Converter para DataFrame
df_results = pd.DataFrame(results, columns=['Nº Componentes', 'Acurácia Treino', 'Acurácia Teste', 'Tempo Execução'])

# Exibir resultados
print(df_results)

   Nº Componentes  Acurácia Treino  Acurácia Teste  Tempo Execução
0               1         0.477149        0.431625        0.297083
1               2         0.590180        0.546318        0.291064
2               5         0.830522        0.763488        0.334076
3              10         0.859494        0.769936        0.442100
4              50         0.891186        0.772650        1.090573


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### Acurácia
 

* A **acurácia** aumentou conforme o número de componentes aumentou, indicando que o modelo aprendeu mais os padrões dos dados.
   
* Contudo, após certo ponto **(5 componentes)**, começou a diminuir a **acurácia**, sugerindo possível **overfitting**.

### Tempo de processamento

* O tempo de execução aumentou conforme o número de componentes cresceu, isso é o comportamento experado pois se tem maior dados a serem calculados